<a href="https://colab.research.google.com/github/gnidetsanna/python_for_ML_tasks/blob/main/HW_2_9_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F_Hnidets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

###Завдання 1###
 Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install opendatasets --upgrade --quiet
!pip install pyarrow --quiet
!pip install --upgrade category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.6 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_curve, auc, precision_score, recall_score, roc_auc_score, r2_score
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [159]:
link_to = '/content/drive/MyDrive/Homeworks/customer_segmentation_train.csv'
raw_df = pd.read_csv(link_to)
raw_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [160]:
unique_counts = raw_df.nunique()
data_types = raw_df.dtypes
is_null = round((raw_df.isnull().sum()/raw_df.shape[0])*100, 2)

df2 = pd.DataFrame({
    'Data Type': data_types,
    'Unique Values': unique_counts,
    'IS_NULL': is_null
})
df2

,Data Type,Unique Values,IS_NULL
ID,int64,8068,0.00
Gender,object,2,0.00
Ever_Married,object,2,1.74
Age,int64,67,0.00
Graduated,object,2,0.97
Profession,object,9,1.54
Work_Experience,float64,15,10.28
Spending_Score,object,3,0.00
Family_Size,float64,9,4.15
Var_1,object,7,0.94


In [161]:
cols = list(raw_df[['Ever_Married','Graduated','Profession','Work_Experience','Family_Size','Var_1']])

In [162]:
raw_df[cols].isna().sum()

Ever_Married       140
Graduated           78
Profession         124
Work_Experience    829
Family_Size        335
Var_1               76
dtype: int64

In [163]:
# Ever Married
raw_df['Ever_Married'].fillna(raw_df['Ever_Married'].mode().iloc[0], inplace=True)

# Graduated
raw_df['Graduated'].fillna('Unknown', inplace=True)

# Profession
raw_df['Profession'].fillna('Unknown', inplace=True)

# Work_Experience
raw_df['Work_Experience'].fillna(raw_df['Work_Experience'].mode().iloc[0], inplace=True)

# Family_Size
raw_df['Family_Size'].fillna(raw_df['Family_Size'].mode().iloc[0], inplace=True)

# Var_1
raw_df['Var_1'].fillna('Unknown', inplace=True)

In [164]:
label_encoder = LabelEncoder()
raw_df['Ever_Married'] = label_encoder.fit_transform(raw_df['Ever_Married'])
raw_df['Graduated'] = label_encoder.fit_transform(raw_df['Graduated'])
raw_df['Gender'] = label_encoder.fit_transform(raw_df['Gender'])
raw_df['Spending_Score'] = label_encoder.fit_transform(raw_df['Spending_Score'])

In [165]:
target_col_name = 'Segmentation'
X = raw_df.drop(columns=target_col_name)
y = raw_df[target_col_name]

X = X.drop(columns=["ID"])

# Розділимо дані на тренувальні та тестові набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [166]:
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

In [167]:
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, cat_cols),
    ])


train_upd = preprocessor.fit_transform(X_train)
test_upd = preprocessor.transform(X_test)

train_upd_df = pd.DataFrame(train_upd, columns=preprocessor.get_feature_names_out())
test_upd_df = pd.DataFrame(test_upd, columns=preprocessor.get_feature_names_out())

In [169]:
train_upd_df.head()

,num__Gender,num__Ever_Married,num__Age,num__Graduated,num__Work_Experience,num__Spending_Score,num__Family_Size,cat__Profession_Artist,cat__Profession_Doctor,cat__Profession_Engineer,...,cat__Profession_Marketing,cat__Profession_Unknown,cat__Var_1_Cat_1,cat__Var_1_Cat_2,cat__Var_1_Cat_3,cat__Var_1_Cat_4,cat__Var_1_Cat_5,cat__Var_1_Cat_6,cat__Var_1_Cat_7,cat__Var_1_Unknown
0,0.0,0.0,0.197183,1.0,0.642857,1.0,0.000,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,1.0,0.760563,1.0,0.071429,0.0,0.125,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.211268,1.0,0.071429,1.0,0.375,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.422535,1.0,0.000000,0.0,0.625,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.140845,0.0,0.642857,1.0,0.000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [176]:
test_upd_df.head()

,num__Gender,num__Ever_Married,num__Age,num__Graduated,num__Work_Experience,num__Spending_Score,num__Family_Size,cat__Profession_Artist,cat__Profession_Doctor,cat__Profession_Engineer,...,cat__Profession_Marketing,cat__Profession_Unknown,cat__Var_1_Cat_1,cat__Var_1_Cat_2,cat__Var_1_Cat_3,cat__Var_1_Cat_4,cat__Var_1_Cat_5,cat__Var_1_Cat_6,cat__Var_1_Cat_7,cat__Var_1_Unknown
0,1.0,1.0,0.323944,0.0,0.857143,0.0,0.250,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.042254,0.0,0.285714,1.0,0.375,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.253521,1.0,0.000000,1.0,0.000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.126761,0.0,0.000000,1.0,0.125,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000,1.0,0.625,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


###Завдання 2###
Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках і порівняти лін регресію на апсампл даних без категоріальних ознак.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [170]:
names = preprocessor.get_feature_names_out()
smote_cols = [name for name in names if 'num_' in name]
smotec_ind = [index for index, col in enumerate(train_upd_df.columns) if 'cat_' in col]

In [171]:
from imblearn.over_sampling import SMOTE, SMOTENC

smote = SMOTE(random_state=42)
smotenc = SMOTENC(random_state=42, categorical_features=smotec_ind)

X_train_smote, y_train_smote = smote.fit_resample(train_upd_df[smote_cols], y_train)

X_train_smotenc, y_train_smotenc = smotenc.fit_resample(train_upd_df, y_train)


display(X_train_smote)
display(X_train_smotenc)

,num__Gender,num__Ever_Married,num__Age,num__Graduated,num__Work_Experience,num__Spending_Score,num__Family_Size
0,0.0,0.0,0.197183,1.0,0.642857,1.0,0.000000
1,1.0,1.0,0.760563,1.0,0.071429,0.0,0.125000
2,0.0,0.0,0.211268,1.0,0.071429,1.0,0.375000
3,0.0,1.0,0.422535,1.0,0.000000,0.0,0.625000
4,0.0,1.0,0.140845,0.0,0.642857,1.0,0.000000
...,...,...,...,...,...,...,...
7251,0.0,1.0,0.731553,1.0,0.000000,0.5,0.125000
7252,1.0,0.0,0.169014,1.0,0.274932,1.0,0.446934
7253,1.0,1.0,0.381115,0.0,0.043702,0.0,0.375000
7254,1.0,1.0,0.323944,1.0,0.071429,0.0,0.125000


,num__Gender,num__Ever_Married,num__Age,num__Graduated,num__Work_Experience,num__Spending_Score,num__Family_Size,cat__Profession_Artist,cat__Profession_Doctor,cat__Profession_Engineer,...,cat__Profession_Marketing,cat__Profession_Unknown,cat__Var_1_Cat_1,cat__Var_1_Cat_2,cat__Var_1_Cat_3,cat__Var_1_Cat_4,cat__Var_1_Cat_5,cat__Var_1_Cat_6,cat__Var_1_Cat_7,cat__Var_1_Unknown
0,0.0,0.0,0.197183,1.0,0.642857,1.0,0.000000,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,1.0,0.760563,1.0,0.071429,0.0,0.125000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.211268,1.0,0.071429,1.0,0.375000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.422535,1.0,0.000000,0.0,0.625000,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.140845,0.0,0.642857,1.0,0.000000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7251,0.0,1.0,0.758038,1.0,0.000000,0.5,0.125000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7252,1.0,0.0,0.241962,1.0,0.480459,1.0,0.303066,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7253,1.0,1.0,0.457655,0.0,0.071429,0.0,0.326479,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7254,1.0,1.0,0.323944,1.0,0.071429,0.0,0.366233,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


###Завдання 3###
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [183]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [173]:
smote_model = OneVsRestClassifier(LogisticRegression())
smotenc_model = OneVsRestClassifier(LogisticRegression())

smote_model.fit(X_train_smote, y_train_smote)
smotenc_model.fit(X_train_smotenc, y_train_smotenc)


OneVsRestClassifier(estimator=LogisticRegression())

In [180]:
smote_prediction = smote_model.predict(test_upd_df[smote_cols])
smotenc_prediction = smotenc_model.predict(test_upd_df)

display(smote_prediction)

array(['A', 'D', 'A', ..., 'A', 'D', 'C'], dtype='<U1')

In [181]:
smote_report = classification_report(y_test, smote_prediction)
smotenc_report = classification_report(y_test, smotenc_prediction)

print("smote report:")
print(smote_report)
print("smotenc report:")
print(smotenc_report)

smote report:
              precision    recall  f1-score   support

           A       0.38      0.36      0.37       394
           B       0.29      0.10      0.14       372
           C       0.47      0.62      0.53       394
           D       0.57      0.74      0.64       454

    accuracy                           0.47      1614
   macro avg       0.42      0.45      0.42      1614
weighted avg       0.43      0.47      0.43      1614

smotenc report:
              precision    recall  f1-score   support

           A       0.42      0.48      0.45       394
           B       0.41      0.25      0.31       372
           C       0.51      0.60      0.55       394
           D       0.68      0.72      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



### Висновок 3 ###
SMOTENC має кращу accuracy, f1 score по кожному окремому класу вищий.
Можливо, модель SMOTENC краща (не набагато).
Якщо ми відштовхуємся від обробки класів - слід звернути увагу на macro_avg - що такой вищий у SMONTENC.

Можна було б спробувати підхід OneVsOne - а ні)
Або погано дані обробили
або потрібно шукати інший підхід побудови моделі, проаналізувавши дизбаланс

In [184]:
ovo_model = OneVsOneClassifier(LogisticRegression())
ovo_model.fit(X_train_smotenc, y_train_smotenc)
ovo_predictions = ovo_model.predict(test_upd_df)

print(classification_report(y_test, ovo_predictions))

              precision    recall  f1-score   support

           A       0.43      0.44      0.44       394
           B       0.40      0.32      0.36       372
           C       0.52      0.56      0.54       394
           D       0.66      0.72      0.69       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.51      1614
weighted avg       0.51      0.52      0.52      1614

